# Data Cleaning Script for Olympic Athletes Dataset

This notebook walks through the process of cleaning the "bios.csv" dataset, covering data inspection, transformation, and cleaning steps.

## 1. Import Required Libraries


In [196]:
# Importing necessary libraries for data processing and visualization

import pandas as pd  # For data manipulation and analysis, especially for working with DataFrames
import numpy as np  # For numerical computations and handling arrays
import seaborn as sns  # For data visualization, providing a high-level interface for drawing attractive graphs
import matplotlib.pyplot as plt  # For plotting and visualizations, a versatile plotting library
from sklearn.impute import SimpleImputer  # For handling missing data using the Simple Imputer with most frequent strategy
import time  # Time library, used for handling time-related tasks
import re  # Regular expressions library, used for string manipulation and pattern matching
import os  # Provides functionalities to interact with the operating system, like reading file paths

## 2. Load the Dataset


In [197]:
# Get the current working directory
pwd = os.getcwd()  # Storing the current directory path

pd.set_option("display.max_columns", None)  # Display all columns in the DataFrame
pd.set_option("display.max_rows", None)     # Display all rows in the DataFrame

# Load the uncleaned dataset into a pandas DataFrame
dataset = pd.read_csv(pwd + "/bios.csv")  # Reading the CSV file into a DataFrame

# Create a copy of the original dataset to work on
df = dataset.copy()  # Making a copy to avoid altering the original data

## 3. Initial Data Inspection
We'll start by inspecting the dataset to understand its structure and content.

In [ ]:
# Display 5 random samples from the dataset
df.sample(5)  # Randomly displaying 5 rows to get a glimpse of the data

In [ ]:
# Display the first 5 rows of the dataset
df.head()  # Showing the first few rows of the DataFrame

In [ ]:
# Display the last 5 rows of the dataset
df.tail()  # Showing the last few rows of the DataFrame

In [ ]:
# Show the shape of the dataset (number of rows and columns)
df.shape  # Getting the dimensions of the DataFrame

In [ ]:
# Show the basic information (data types, non-null counts, etc.) of the dataset
df.info()  # Displaying information about the DataFrame's structure

In [ ]:
# Show the number of missing values for each column
df.isnull().sum()  # Counting missing values in each column

In [ ]:
df.isnull().sum() / df.shape[0] * 100  # Calculate the percentage of missing values in each column

In [ ]:
# Show the number of duplicate rows in the dataset
df.duplicated().sum()  # Checking for duplicate entries

In [206]:
# Drop duplicate rows, if any, to avoid redundancy
df = df.drop_duplicates()  # Removing any duplicate records

In [207]:
# First step of dropping unwanted columns for clean summary
df = df.drop(columns=["Nick/petnames","Title(s)","Other names",
                      "Nationality","Original name","Name order"])

In [ ]:
# Display the list of column names in the DataFrame
df.columns

In [209]:
# Rearranging columns for better organization
df = df[['athlete_id','Full name', 'Used name','Sex','Born',
          'Died', 'NOC', 'Measurements', 'Roles','Affiliations']]

In [ ]:
# Randomly sample a single row from the rearranged DataFrame for a quick overview
df.sample()

## 4. Summary Statistics

Let's inspect summary statistics for both numeric and categorical columns.


In [ ]:
# Loop through all object type columns and display their value counts
for i in df.select_dtypes(include="object").columns:  # Iterating over object columns
    print(f"Value Counts for {i}:\n", df[i].value_counts())  # Displaying counts of unique values in each column
    print("***" * 10)  # Separator for clarity

In [ ]:
# Display summary statistics for object (categorical) columns
df.describe(include="object")  # Summary statistics for categorical variables

## 5. Data Cleaning for Used Name Column

In this section, we explore the "Used name" column, clean it by replacing unwanted characters, and then drop unnecessary columns.

In [ ]:
df.columns # Displays the column names of the DataFrame

In [ ]:
# List unique values in the "Used name" column
list(df["Used name"].unique())  # Outputs a list of unique values in the "Used name" column

In [ ]:
# Display summary statistics for the "Used name" column
df["Used name"].describe()  # Provides summary statistics for the "Used name" column

In [216]:
# Insert a new column "name" after the "Used name" column, replacing "•" with a space
df.insert(loc=df.columns.get_loc("Used name")+1, column="name", value=df["Used name"].str.replace("•", " "))  
# Adds a new "name" column, with '•' replaced by spaces

In [ ]:
# Display a random sample of 3 rows from the DataFrame
df.sample(3)  # Displays 3 random rows from the DataFrame

In [218]:
# Drop the "Full name" and "Used name" columns from the DataFrame
df = df.drop(columns=["Full name", "Used name"])  # Removes the "Full name" and "Used name" columns

In [ ]:
# Display another random sample of 3 rows from the DataFrame
df.sample(3)  # Displays 3 random rows from the updated DataFrame

## 6. Exploring the Sex Column

In this section, we analyze the "Sex" column to understand its unique values, descriptive statistics, and distribution through visual representation.


In [ ]:
# Display unique values in the "Sex" column
df["Sex"].unique()  # Outputs an array of unique values in the "Sex" column

In [ ]:
# Display summary statistics for the "Sex" column
df["Sex"].describe()  # Provides summary statistics for the "Sex" column

In [ ]:
# Count the occurrences of each unique value in the "Sex" column
df["Sex"].value_counts()  # Returns a Series with counts of unique values in the "Sex" column

In [ ]:
# Create a bar plot of the value counts for the "Sex" column
df["Sex"].value_counts().plot.bar()  # Generates a bar plot to visualize the distribution of values in the "Sex" column

## 7. Data Cleaning and Transformation for the Born Column

In this section, we clean and transform the "Born" column to extract birth date and location details.

In [ ]:
# List unique values in the "Born" column
list(df["Born"].unique())  # Outputs a list of unique values in the "Born" column

In [ ]:
# Count occurrences of each unique value in the "Born" column
df["Born"].value_counts()  # Returns a Series with counts of unique values in the "Born" column

In [226]:
# Define new column names for birth date and location
new_cols1 = ["birth_date", "birth_location"]  # Names for the new columns

In [227]:
# Split the 'Born' column into two new columns based on the word 'in'
new_data1 = df["Born"].str.split("in", expand=True)  # Splits the column into two parts

In [228]:
# Replace month names with their respective numeric values
new_data1[0] = new_data1[0].replace({"January":"-01-","February":"-02-","March":"-03-","April":"-04-",
                                      "May":"-05-","June":"-06-","July":"-07-","August":"-08-",
                                      "September":"-09-","October":"-10-","November":"-11-","December":"-12-"}, regex=True)  

In [229]:
# Strip leading and trailing spaces from birth_date and birth_location columns
new_data1[0] = new_data1[0].str.strip().replace(" ", "", regex=False)  # Cleans the birth_date
new_data1[1] = new_data1[1].str.strip().replace(" ", "", regex=False)  # Cleans the birth_location

In [230]:
# Insert the new columns into the DataFrame
for i in range(len(new_cols1)):
    df.insert(loc=df.columns.get_loc("Born") + 1 + i, column=new_cols1[i], value=new_data1[i])  # Adds new columns

In [ ]:
# Display a random sample of 3 rows from the updated DataFrame
df.sample(3)  # Outputs a random sample of 3 rows

In [232]:
# Remove spaces within the 'birth_date' strings
df["birth_date"] = df["birth_date"].str.replace(" ", "", regex=False)  # Cleans the birth_date values

# Convert 'birth_date' to datetime format after cleaning
df["birth_date"] = pd.to_datetime(df["birth_date"], format="%d-%m-%Y", errors='coerce')  # Converts to datetime


In [ ]:
# Display another random sample of 3 rows from the updated DataFrame
df.sample(3)  # Outputs a random sample of 3 rows

In [234]:
# Remove spaces within the 'birth_location' strings
df["birth_location"] = df["birth_location"].str.replace(" ", "", regex=False)  # Cleans the birth_location values

In [235]:
# Define new column names for City and Region_country
new_cols2 = ["City", "Region_country"]  # Names for the new columns

In [236]:
# Split the 'birth_location' column into two new columns based on the comma
new_data2 = df["birth_location"].str.split(",", expand=True)  # Splits the column into two parts

In [237]:
# Insert the new columns into the DataFrame
for i in range(len(new_cols2)):
    df.insert(loc=df.columns.get_loc("birth_location") + 1 + i, column=new_cols2[i], value=new_data2[i])  # Adds new columns

In [ ]:
# Display a random sample from the updated DataFrame
df.sample()  # Outputs a random sample of rows

In [239]:
# Define new column names for Region and Country
new_cols3 = ["Region", "Country"]  # Names for the new columns

In [240]:
# Split the 'Region_country' column into two new columns based on the opening parenthesis
new_data3 = df["Region_country"].str.split("(", expand=True)  # Splits the column into two parts

In [241]:
# Insert the new columns into the DataFrame
for i in range(len(new_cols3)):
    df.insert(loc=df.columns.get_loc("Region_country") + 1 + i, column=new_cols3[i], value=new_data3[i])  # Adds new columns

In [ ]:
# Display a random sample from the updated DataFrame
df.sample()  # Outputs a random sample of rows

In [243]:
# Remove the closing parenthesis ')' from the 'Country' column
df["Country"] = df["Country"].str.replace(")", "", regex=False)  # Cleans the Country values

In [ ]:
# Display a random sample from the updated DataFrame
df.sample()  # Outputs a random sample of rows

In [245]:
# Drop the original 'Born', 'birth_location', 'Region_country', and 'Died' columns from the DataFrame
df = df.drop(columns=["Born", "birth_location", "Region_country", "Died"])  # Removes specified columns

In [ ]:
# Display a random sample from the updated DataFrame
df.sample()  # Outputs a random sample of rows

## 8. NOC Column Inspection and Cleaning

In this section, we inspect the "NOC" (National Olympic Committee) column for unique values, descriptions, and missing values, then clean the data.

In [ ]:
# Display unique values in the "NOC" column
df["NOC"].unique()  # Outputs a list of unique values in the "NOC" column

In [ ]:
# Display descriptive statistics for the "NOC" column
df["NOC"].describe()  # Returns descriptive statistics for the "NOC" column

In [ ]:
# Count occurrences of each unique value in the "NOC" column
df["NOC"].value_counts()  # Returns a Series with counts of unique values in the "NOC" column

In [ ]:
# Identify rows where "NOC" is NaN (missing)
nan_in_noc = df["NOC"].isna()  # Creates a boolean Series indicating missing values
df[nan_in_noc]  # Outputs the rows where the "NOC" column has missing values

In [251]:
# Drop rows where "NOC" is NaN
df.dropna(subset=["NOC"], inplace=True)  # Removes rows with NaN in the "NOC" column

## 9. Measurements Column Inspection and Cleaning

In this section, we inspect the "Measurements" column for unique values, descriptions, and then clean the data to extract height and weight.

In [ ]:
# Display unique values in the "Measurements" column
df["Measurements"].unique()  # Outputs unique values found in the "Measurements" column

In [ ]:
# Count occurrences of each unique value in the "Measurements" column
df["Measurements"].value_counts()  # Returns a Series with counts of unique values in the "Measurements" column

In [ ]:
# Display descriptive statistics for the "Measurements" column
df["Measurements"].describe()  # Returns descriptive statistics for the "Measurements" column

In [255]:
# Define new columns for height and weight
new_cols4 = ["height_cm", "weight_kg"]  # List of new column names

In [256]:
# Split 'Measurements' column into two new columns; will result in NaN if there's no second part
new_data4 = df["Measurements"].str.split("/", expand=True)  # Split the measurements into separate columns

In [257]:
# Strip any whitespace from the split columns
new_data4 = new_data4.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # Remove whitespace

# Initialize new columns in the DataFrame
df["height_cm"] = None  # Create a column for height
df["weight_kg"] = None  # Create a column for weight

# Handle cases where there's only one value (either height or weight)
for i, row in new_data4.iterrows():  # Iterate over rows of new data
    if pd.isna(row[1]):  # If the second column is NaN (indicating a single value)
        if pd.notna(row[0]):  # Check if the single value is valid
            if "cm" in row[0]:  # Check if the value contains "cm"
                df.at[i, "height_cm"] = row[0].replace(" cm", "")  # Store height
            elif "kg" in row[0]:  # Check if the value contains "kg"
                df.at[i, "weight_kg"] = row[0].replace(" kg", "")  # Store weight
    else:
        # If both height and weight are present, assign them to the respective columns
        if pd.notna(row[0]):  # Check if height is valid
            df.at[i, "height_cm"] = row[0].replace(" cm", "")  # Store height
        if pd.notna(row[1]):  # Check if weight is valid
            df.at[i, "weight_kg"] = row[1].replace(" kg", "")  # Store weight

In [258]:
# Replace "cm" and "kg" in the split data
new_data4 = new_data4.replace({"cm": "", "kg": ""}, regex=True)  # Remove units from data

In [259]:
# Convert columns to numeric values
df["height_cm"] = pd.to_numeric(df["height_cm"], errors="coerce")  # Convert height to numeric, coercing errors
df["weight_kg"] = pd.to_numeric(df["weight_kg"], errors="coerce")  # Convert weight to numeric, coercing errors

In [ ]:
# Display a sample of the DataFrame after transformations
df.sample()  # Outputs a random sample of the DataFrame

In [261]:
# Drop the original "Measurements" column as it is no longer needed
df = df.drop(columns=["Measurements"])  # Remove the "Measurements" column from the DataFrame

In [262]:
# Rearrange the DataFrame to show only relevant columns
df = df[['athlete_id', 'name', 'Sex', 'birth_date', 'City', 'Region', 'Country',
       'NOC','height_cm','weight_kg', 'Roles', 'Affiliations']]  # Select specific columns to keep

In [ ]:
# Display a sample of the DataFrame after rearranging columns
df.sample()  # Outputs a random sample of the DataFrame after column rearrangement

## 10. Handling Missing Values and Final Data Cleanup

In this section, we check for missing values in the DataFrame, fill missing entries, and apply final transformations.

In [ ]:
# Check for missing values in the DataFrame
df.isna().sum()  # Returns the count of missing values for each column

In [ ]:
# Display the names of columns in the DataFrame
df.columns  # Outputs the list of column names in the DataFrame

In [266]:
# Fill missing values in specific columns with "Unknown"
df[["City", "Region", "Country", "Affiliations"]] = df[["City", "Region", "Country", "Affiliations"]].fillna("Unknown")  # Replace NaN with "Unknown"

In [ ]:
# Check for missing values again after filling
df.isna().sum()  # Returns the count of missing values for each column

In [ ]:
# Display information about the DataFrame, including data types and non-null counts
df.info()  # Outputs a concise summary of the DataFrame

In [269]:
# Initialize SimpleImputer with the strategy set to "most_frequent"
imputer = SimpleImputer(strategy="most_frequent")  # Create an imputer for filling missing values with the most frequent value

# Iterate through numeric columns and fill missing values
for i in df.select_dtypes(include="number").columns:  # Loop through numeric columns
    df[i] = imputer.fit_transform(df[[i]])  # Fill missing values in each numeric column

In [ ]:
# Fill missing birth_date with a constant value
df["birth_date"].fillna(pd.Timestamp("1975-01-01"), inplace=True)  # Replace NaN in 'birth_date' with a specified date

In [271]:
# Round the height and weight columns to one decimal place
df["height_cm"] = df["height_cm"].round(1)  # Round height to one decimal place
df["weight_kg"] = df["weight_kg"].round(1)  # Round weight to one decimal place

In [ ]:
# Check for any remaining missing values in the DataFrame
df.isna().sum()  # Returns the count of missing values for each column

In [ ]:
# Display a random sample of 3 rows from the DataFrame
df.sample(3)  # Outputs a random sample of 3 rows from the DataFrame

## 11. Final Output
In this section, we'll create a copy of the cleaned DataFrame and save it as a CSV file named `cleaned_bios.csv`. This file will be stored in the current working directory.

In [281]:
# Create a copy of the cleaned DataFrame
output = df.copy()  # Making a copy of the DataFrame to preserve the cleaned data

In [282]:
# Save the cleaned DataFrame to a CSV file
output.to_csv(pwd + "/cleaned_bios.csv", index=False)  # Saving the DataFrame to a CSV without the index